APRD 6342

Samuel Statton

Python Exercise 4

##Parameters for Organic Coffee

1: Income between 50k and 74.5k

2: Age betwee 18 and 34

3: Hispanic

In [0]:
import pandas as pd
import numpy as np
import requests
import json

In [0]:
msas = pd.read_csv("https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv")
msas.head()

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


In [0]:
tableids = ['B03002', 'B01001', 'B19001']

In [0]:
tablestring = ','.join(tableids)

In [0]:
requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % tablestring

In [0]:
url = requesturl

In [0]:
loadedjson = requests.get(url)

In [0]:
parsedjson = loadedjson.json()

In [0]:
## failed function ##
def fun(d):
    if 'B19001' in d:
        data = d.values
    for k in d:
        if isinstance(d[k], list):
            for i in d[k]:
                for j in fun(i):
                    yield j
fun(parsedjson)

In [0]:
income = pd.DataFrame({ 'income_50k': { id_: data['B19001']['estimate']['B19001011'] for id_, data in parsedjson['data'].items() }})
hispanic = pd.DataFrame({ 'hispanic': { id_: data['B03002']['estimate']['B03002012'] for id_, data in parsedjson['data'].items() }})
age = pd.DataFrame({ '18 to 34 years': { id_: data['B01001']['estimate']['B01001001'] for id_, data in parsedjson['data'].items() }})


In [0]:
df = income.merge(hispanic, how='outer',left_index=True, right_index=True)
df = df.merge(age, how='outer', left_index=True, right_index=True)
df.head()

,income_50k,hispanic,18 to 34 years
31000US10100,1479.0,1020.0,42608.0
31000US10140,1938.0,6978.0,71454.0
31000US10180,5409.0,38892.0,169000.0
31000US10220,1258.0,1944.0,38289.0
31000US10260,192.0,18459.0,18525.0


##Question 1:


In [0]:
df.sort_values(by='income_50k', ascending=False).head()

,income_50k,hispanic,18 to 34 years
31000US35620,444453.0,4863602.0,20192042.0
31000US31080,300860.0,5972751.0,13261538.0
31000US16980,250089.0,2081524.0,9549229.0
31000US19100,199823.0,2018504.0,7104415.0
31000US26420,172121.0,2432651.0,6636208.0


In [0]:
msas.loc[msas['CBSA'] == 35620, 'NAME']

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object

New York-Newark-Jersey City is the highest population with households between 50k and 74.9k.

##Question 2:

In [0]:
df.sort_values(by='hispanic', ascending=False).head()

,income_50k,hispanic,18 to 34 years
31000US31080,300860.0,5972751.0,13261538.0
31000US35620,444453.0,4863602.0,20192042.0
31000US33100,158933.0,2663235.0,6019790.0
31000US26420,172121.0,2432651.0,6636208.0
31000US40140,104313.0,2239029.0,4476222.0


In [0]:
msas.loc[msas['CBSA'] == 31080, 'NAME']

210    Los Angeles-Long Beach-Anaheim, CA
Name: NAME, dtype: object

Los Angeles-Long Beach-Anaheim is the city with the most Hispanics

##Question 3:

In [0]:
df.sort_values(by='18 to 34 years', ascending=False).head()

,income_50k,hispanic,18 to 34 years
31000US35620,444453.0,4863602.0,20192042.0
31000US31080,300860.0,5972751.0,13261538.0
31000US16980,250089.0,2081524.0,9549229.0
31000US19100,199823.0,2018504.0,7104415.0
31000US26420,172121.0,2432651.0,6636208.0


In [0]:
msas.loc[msas['CBSA'] == 35620, 'NAME']

249    New York-Newark-Jersey City, NY-NJ-PA
Name: NAME, dtype: object

New York-Newark-Jersey city has the msot people 18-34 years old.

## Question 4:

In [0]:
df['income'] = ''
df['ethnicity'] = ''
df['age'] = ''
df['mean_parameter'] = ''

for i in range(len(df)):
  df['income'].values[i] = df.iloc[i,0]/sum(df.iloc[:,0])
  df['ethnicity'].values[i] = df.iloc[i,1]/sum(df.iloc[:,1])
  df['age'].values[i] = df.iloc[i,2]/sum(df.iloc[:,2])

df['mean_parameter'] = df[['income','ethnicity','age']].mean(axis=1)

In [0]:
df.sort_values(by='mean_parameter',ascending=False)

,income_50k,hispanic,18 to 34 years,income,ethnicity,age,mean_parameter
31000US35620,444453.0,4863602.0,20192042.0,0.0514338,0.0828389,0.0659695,0.066747
31000US31080,300860.0,5972751.0,13261538.0,0.0348167,0.10173,0.0433268,0.059958
31000US16980,250089.0,2081524.0,9549229.0,0.0289413,0.0354534,0.0311983,0.031864
31000US33100,158933.0,2663235.0,6019790.0,0.0183923,0.0453613,0.0196673,0.027807
31000US26420,172121.0,2432651.0,6636208.0,0.0199185,0.0414339,0.0216812,0.027678
...,...,...,...,...,...,...,...
31000US18780,356.0,1948.0,13056.0,4.11977e-05,3.31792e-05,4.26553e-05,0.000039
31000US28540,513.0,655.0,13745.0,5.93663e-05,1.11562e-05,4.49063e-05,0.000038
31000US47240,435.0,280.0,17275.0,5.03399e-05,4.76908e-06,5.64392e-05,0.000037
31000US32500,442.0,126.0,17167.0,5.11499e-05,2.14609e-06,5.60864e-05,0.000036


New York-Neward-New Jersey is the city with the highest mean followed by Los Angeles